In [1]:
!pip install pandasql

In [2]:
!pip install dash

In [3]:
import pandas as pd
import pandasql as ps
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import html
from dash import dcc
from dash import Input
from dash import Output

In [4]:
dataset = pd.read_csv('retailSusu.csv',sep=';')

In [5]:
dataset

,orderDate,city,province,product,qty,price
0,2022-10-17,Jakarta Selatan,DKI Jakarta,SUSU_J,10,7400
1,2022-10-24,Jakarta Selatan,DKI Jakarta,SUSU_R,2,6040
2,2022-11-03,Jakarta Utara,DKI Jakarta,SUSU_C,8,10450
3,2022-08-19,Jakarta Barat,DKI Jakarta,SUSU_A,4,2050
4,2022-10-16,Jakarta Timur,DKI Jakarta,SUSU_R,2,44750
...,...,...,...,...,...,...
4995,2022-12-01,Tangerang,Banten,SUSU_R,2,4500
4996,2022-07-28,Bogor,Jawa Barat,SUSU_C,3,14650
4997,2022-10-23,Jakarta Timur,DKI Jakarta,SUSU_H,4,7470
4998,2022-10-17,Jakarta Barat,DKI Jakarta,SUSU_C,8,6950


In [6]:
dataset['gmv'] = dataset['qty'] * dataset['price']
dataset

,orderDate,city,province,product,qty,price,gmv
0,2022-10-17,Jakarta Selatan,DKI Jakarta,SUSU_J,10,7400,74000
1,2022-10-24,Jakarta Selatan,DKI Jakarta,SUSU_R,2,6040,12080
2,2022-11-03,Jakarta Utara,DKI Jakarta,SUSU_C,8,10450,83600
3,2022-08-19,Jakarta Barat,DKI Jakarta,SUSU_A,4,2050,8200
4,2022-10-16,Jakarta Timur,DKI Jakarta,SUSU_R,2,44750,89500
...,...,...,...,...,...,...,...
4995,2022-12-01,Tangerang,Banten,SUSU_R,2,4500,9000
4996,2022-07-28,Bogor,Jawa Barat,SUSU_C,3,14650,43950
4997,2022-10-23,Jakarta Timur,DKI Jakarta,SUSU_H,4,7470,29880
4998,2022-10-17,Jakarta Barat,DKI Jakarta,SUSU_C,8,6950,55600


In [7]:
dataset['order_month'] = dataset['orderDate'].str.slice(0,7)
dataset

,orderDate,city,province,product,qty,price,gmv,order_month
0,2022-10-17,Jakarta Selatan,DKI Jakarta,SUSU_J,10,7400,74000,2022-10
1,2022-10-24,Jakarta Selatan,DKI Jakarta,SUSU_R,2,6040,12080,2022-10
2,2022-11-03,Jakarta Utara,DKI Jakarta,SUSU_C,8,10450,83600,2022-11
3,2022-08-19,Jakarta Barat,DKI Jakarta,SUSU_A,4,2050,8200,2022-08
4,2022-10-16,Jakarta Timur,DKI Jakarta,SUSU_R,2,44750,89500,2022-10
...,...,...,...,...,...,...,...,...
4995,2022-12-01,Tangerang,Banten,SUSU_R,2,4500,9000,2022-12
4996,2022-07-28,Bogor,Jawa Barat,SUSU_C,3,14650,43950,2022-07
4997,2022-10-23,Jakarta Timur,DKI Jakarta,SUSU_H,4,7470,29880,2022-10
4998,2022-10-17,Jakarta Barat,DKI Jakarta,SUSU_C,8,6950,55600,2022-10


In [8]:
city_uniques = dataset['city'].unique()
city_uniques

array(['Jakarta Selatan', 'Jakarta Utara', 'Jakarta Barat',
       'Jakarta Timur', 'Surakarta', 'Tangerang', 'Denpasar', 'Surabaya',
       'Jakarta Pusat', 'Bogor', 'Bandung', 'Yogyakarta', 'Malang',
       'Bekasi', 'Depok', 'Semarang', 'Purwokerto', 'Ubud'], dtype=object)

In [9]:
monthly_order = dataset.groupby(['order_month','city',])['gmv'].sum().reset_index()
monthly_order

,order_month,city,gmv
0,2022-07,Bandung,889340
1,2022-07,Bekasi,1502740
2,2022-07,Bogor,1509770
3,2022-07,Denpasar,767100
4,2022-07,Depok,294300
...,...,...,...
99,2022-12,Surabaya,2058390
100,2022-12,Surakarta,5970790
101,2022-12,Tangerang,4434660
102,2022-12,Ubud,104500


In [10]:
app = dash.Dash()

app.layout = html.Div(children = [html.H1('Creativ Dev Indonesia', style={'textAlign':'center', 'color':'#000000'}),
                                  html.P('penjualan susu', style={'textAlign':'center'}),
                                  dcc.Dropdown(
                                      id='myDropdown',
                                      options=[{'label':k, 'value':k} for k in city_uniques],
                                      value=['Jakarta Selatan', 'Jakarta Barat', 'Bandung', 'Depok'],
                                      multi=True),
                                  html.Hr(),
                                  dcc.Graph(id='display-milk')
                                  ])
@app.callback(Output('display-milk',component_property='figure'),
              Input('myDropdown', component_property='value'))

def for_update_input(value):
    data_susu = monthly_order[monthly_order["city"].isin(value)]
    fig = px.line(data_susu, x ="order_month", y ="gmv", color="city")
    return fig

if __name__ == '__main__':
    app.run_server()